<a href="https://colab.research.google.com/github/ggesa432/ggesa432/blob/master/ag7432_deep_learning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRSAS: Consolidated Recommendation and Sentiment Analysis System

## Team Members: Yams Gupta, Zeren Gesang

## Specific Goals
Sentiment Analysis: Assess the sentiment of user reviews to determine overall customer
satisfaction and perception.

Recommendation System: Develop a system that recommends businesses or products based on user preferences and sentiment scores.

## Datasets:

- 20newsgroup dataset
- IMDB Reviews dataset

## Methodology
Data Preprocessing, Cleaning and Structuring: Address missing values, remove noise, and
structure the data for analysis.

Feature Engineering: Extract features relevant to sentiment and recommendation, like review text, ratings, user activity, etc.

Deep Learning Models
RoBERTa for Sentiment Analysis: Utilize RoBERTa, a robust transformer-based model, for analyzing the sentiment of reviews.

Analysis and Interpretation
Sentiment Trends: Analyze sentiment trends over time, across different business categories, and geographical areas.

## Link to Repo: https://github.com/DubbleA/deep-learning/tree/main/final-project

This will read like similar to our demo's in class, all technical details and reflection will be integrated with the notebook itself.

### There are 3 Sections, each with its own summary and conclusion! Thank you!

### Project Idea: Case study for Movie Catalog Companies i.e. Disney Plus, HBO Max, Netflix

1. `Topic Classification`: First neural network classifies the text into one of the 20 newsgroups (topics). This type of neural network can be used by a company like Netflix to organize their movies into a catalog using deep learning.

2. `Sentiment Analysis`: Second neural network analyzes the sentiment (positive, negative, neutral) of the text. Netflix or similar can use this to determine if a movie or show has positive or negative ratings, and more accurately guage its feedback.

We'll use the 20 Newsgroups dataset for topic classification and a simple sentiment analysis dataset for the second part.

In [ ]:
#Step 1: Import Necessary Libraries
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#Step 2: Load and Preprocess the 20 Newsgroups Dataset
# Load the dataset
newsgroups_data = fetch_20newsgroups(subset='all')
texts = newsgroups_data.data
labels_tensor = newsgroups_data.target

# Tokenization and Encoding (Using RoBERTa tokenizer)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def encode_texts(tokenizer, texts, labels, max_length=512):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, torch.tensor(labels)

input_ids, attention_masks, labels = encode_texts(tokenizer, texts, labels_tensor)

# Splitting the data into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=2018, test_size=0.1)

<ipython-input-6-09457c654e13>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return input_ids, attention_masks, torch.tensor(labels)


In [ ]:
class NewsgroupsDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

train_dataset = NewsgroupsDataset(train_inputs, train_masks, train_labels)
validation_dataset = NewsgroupsDataset(validation_inputs, validation_masks, validation_labels)

# Reduced batch size to alleviate GPU memory issues from 32 to 16
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
validation_loader = DataLoader(validation_dataset, batch_size=16, pin_memory=True)

#Step 4: Build and Train the Topic Classification Model
# Model Initialization
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(newsgroups_data.target_names))

# Define Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

# Checking if GPU is available and then setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Moving the model to the chosen device
model.to(device)

# Training Loop with Gradient Accumulation
epochs = 4
accumulation_steps = 2  # Define how many steps to accumulate gradients for

for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    optimizer.zero_grad()  # Reset gradients at the start of each epoch

    for step, batch in enumerate(train_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss = loss / accumulation_steps  # Normalize our loss (if averaged)
        loss.backward()

        if (step + 1) % accumulation_steps == 0:  # Wait for several backward steps
            optimizer.step()  # Now we can do an optimizer step
            model.zero_grad()  # Reset gradients after optimizer step

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Average training loss: {avg_train_loss}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average training loss: 0.8435588421354476
Average training loss: 0.3089287603953388
Average training loss: 0.1870358764807519
Average training loss: 0.12156452839069673


In [ ]:
#Step 5: Evaluate the Topic Classification Model
# Evaluation Loop
model.eval()
predictions, true_labels = [], []

for batch in validation_loader:
    batch_input_ids = batch['input_ids'].to(device)
    batch_input_mask = batch['attention_mask'].to(device)
    batch_labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(batch_input_ids, token_type_ids=None, attention_mask=batch_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = batch_labels.to('cpu').numpy()
    predictions.append(logits)
    true_labels.append(label_ids)

# Report the final accuracy for this validation run.
predicted_labels = np.concatenate([np.argmax(p, axis=1) for p in predictions])
true_labels = np.concatenate(true_labels)
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89        72
           1       0.86      0.86      0.86        96
           2       0.81      0.80      0.80        84
           3       0.84      0.77      0.81       105
           4       0.94      0.92      0.93       104
           5       0.91      0.95      0.93       121
           6       0.92      0.92      0.92       105
           7       0.98      0.91      0.95       104
           8       0.92      0.94      0.93       112
           9       0.99      0.98      0.98        98
          10       0.97      0.98      0.97        94
          11       0.97      0.97      0.97       101
          12       0.80      0.92      0.85        89
          13       0.98      0.93      0.95       101
          14       0.98      0.95      0.97       104
          15       0.95      0.94      0.95       103
          16       0.96      0.96      0.96        69
          17       0.97    

In [ ]:
def predict_newsgroup_category(text, model, tokenizer):
    # Encode the text using the same tokenizer used during training
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',  # Updated padding
        truncation=True,       # Explicit truncation
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Extract input IDs and attention masks from the encoded text
    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    # Model in evaluation mode
    model.eval()

    # Predict without computing gradients
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)

    # Extract logits
    logits = outputs.logits

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1).cpu().numpy()[0]

    return newsgroups_data.target_names[predicted_class]

# Example usage
sample_text = "NASA launches a new satellite to study star formations."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")

sample_text = "Last night's basketball game went into overtime with an incredible buzzer-beater shot."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")

sample_text = "The latest advancements in quantum computing are set to revolutionize the tech industry."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")

sample_text = "The recent elections have shown a significant shift in regional political dynamics."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")

sample_text = "Exploring the philosophical dimensions of Buddhism and its meditation practices."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")

sample_text = "Debating the merits of electric vehicles versus traditional gasoline-powered cars."
category = predict_newsgroup_category(sample_text, model, tokenizer)
print(f"{sample_text} ... predicted category: {category}")


NASA launches a new satellite to study star formations. ... predicted category: sci.space
Last night's basketball game went into overtime with an incredible buzzer-beater shot. ... predicted category: rec.sport.hockey
The latest advancements in quantum computing are set to revolutionize the tech industry. ... predicted category: sci.space
The recent elections have shown a significant shift in regional political dynamics. ... predicted category: talk.politics.misc
Exploring the philosophical dimensions of Buddhism and its meditation practices. ... predicted category: alt.atheism
Debating the merits of electric vehicles versus traditional gasoline-powered cars. ... predicted category: rec.autos


## Section Summary: Neural Network-Based Classification

### Data and Training
The model was trained on the 20 Newsgroups dataset, which comprises various topics ranging from technology and sports to politics and religion. The training process showed consistent improvement in loss reduction, with the final training loss being reduced to apprx 0.1215.

### Results
The classification model demonstrated high accuracy, with an overall precision, recall, and f1-score of around 92%. The model categorized text samples, such as recognizing a sports event as 'rec.sport.hockey' and a political discussion under 'talk.politics.misc'. Even though these classifications were slightly innacurate they got the general idea correct.

### Application for Movie Catalogs
Topic Classification: This model can automatically categorize movies and shows based on their descriptions into predefined genres or themes. For example, a movie about space exploration can be categorized under 'sci.space', and a political drama could fall under 'talk.politics.misc'. This could be really useful for community driven platform like YouTube to recommend content to its users

# Section 2: Sentiment Analysis Model

In [ ]:
!pip install portalocker torchtext

In [ ]:
#Step 1: Import Necessary Libraries
import portalocker
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torchtext.datasets import IMDB
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.optim

#Step 2: Load and Preprocess the IMDB Dataset
# Use RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
MAX_SEQ_LENGTH = 512

def data_process(data_iter):
    data = []
    for label, text in data_iter:
        encoded_text = tokenizer.encode(text, truncation=True, max_length=MAX_SEQ_LENGTH)
        data.append((label, torch.tensor(encoded_text, dtype=torch.long)))
    return data

train_data = data_process(train_iter)
#valid_data = data_process(IMDB(split='valid'))
test_data = data_process(test_iter)

# Split the training data for validation
train_data, valid_data = train_test_split(train_data, test_size=0.1)

# Example code to inspect a few data samples
for i, (label, text) in enumerate(train_data[:5]):
    print(f"Sample {i+1}:")
    print("Label:", label)
    print("Text:", tokenizer.decode(text))
    print()


Sample 1:
Label: 1
Text: <s>It's been nearly 30 years, and I STILL hate everyone involved in this movie. It remains the worst movie I've ever seen.<br /><br />Before seeing this, I never much minded Rivers, one way or the other. After seeing this movie, I have an allergic reaction when I accidentally see her on television.<br /><br />I got dragged to this - against my better judgment - by peer pressure. However, coming out of the theater, those friends swore an oath to never again overrule my choice of movie. Nearly thirty years later, we still carry around mental scars from this movie.<br /><br />On my deathbed, one of my regrets will be the time I wasted hoping that this movie might get better. It never did.<br /><br />If you are ever given a choice, you would prefer putting your own eyes out to sitting though this movie.<br /><br />I registered for IMDb comments just in the hope that perhaps I can warn others against viewing this movie. If I can save just one person from watching th

In [ ]:
#Step 3: Create PyTorch Dataset and DataLoader for IMDB data.
from torch.nn.utils.rnn import pad_sequence
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"



MAX_SEQ_LENGTH = 512  # Maximum sequence length for RoBERTa

class IMDBDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, text = self.data[idx]
        # Map 2 to 1 (positive) and 1 to 0 (negative)
        mapped_label = 0 if label == 1 else 1
        return text, mapped_label

    def collate_fn(batch):
        texts, labels = zip(*batch)

        # Truncate the texts to MAX_SEQ_LENGTH
        texts = [t[:MAX_SEQ_LENGTH] for t in texts]
        texts = pad_sequence(texts, batch_first=True, padding_value=0)

        labels = torch.tensor(labels)

        # Create attention masks
        attention_masks = (texts != 0).long()

        return texts, attention_masks, labels

# Create instances of IMDBDataset
train_dataset = IMDBDataset(train_data)
valid_dataset = IMDBDataset(valid_data)
test_dataset = IMDBDataset(test_data)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=IMDBDataset.collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=16, collate_fn=IMDBDataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=IMDBDataset.collate_fn)

# Check label distribution in the dataset
train_labels = [label for label, _ in train_data]
valid_labels = [label for label, _ in valid_data]
test_labels = [label for label, _ in test_data]

print("Training label distribution:", np.bincount(train_labels))
print("Validation label distribution:", np.bincount(valid_labels))
print("Test label distribution:", np.bincount(test_labels))

#Step 4: Build and Train the Sentiment Analysis Model
#For sentiment analysis, we'll use a RoBERTa model, similar to the previous example.

# Initialize RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Checking if GPU is available and then setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Moving the model to the chosen device
model.to(device)

# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (text, attention_mask, label) in enumerate(train_loader):

        # Debugging: Print model output and labels for the first batch
        if i == 0:
            print("Epoch:", epoch + 1)
            print("Sample outputs:", outputs.logits[:5])
            print("Sample labels:", label[:5])
        text = text.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        outputs = model(text, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch: {epoch+1}, Average Loss: {avg_loss}")

Training label distribution: [    0 11254 11246]
Validation label distribution: [   0 1246 1254]
Test label distribution: [    0 12500 12500]


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Sample outputs: tensor([[ 6.4435, -6.3981],
        [ 6.5287, -6.2771],
        [ 6.4168, -5.9419],
        [ 6.2167, -6.2937],
        [ 6.4172, -6.2291]], device='cuda:0', grad_fn=<SliceBackward0>)
Sample labels: tensor([1, 1, 1, 1, 1])
Epoch: 1, Average Loss: 0.21402609751373794
Epoch: 2
Sample outputs: tensor([[-0.1156,  0.2722],
        [ 2.5939, -1.9025],
        [ 2.7270, -2.0931],
        [ 2.6023, -1.9946]], device='cuda:0', grad_fn=<SliceBackward0>)
Sample labels: tensor([1, 1, 0, 1, 0])
Epoch: 2, Average Loss: 0.12650287883983716
Epoch: 3
Sample outputs: tensor([[ 2.3323, -1.6680],
        [-0.5708,  1.2392],
        [ 0.5558, -0.1400],
        [ 1.6588, -1.1124]], device='cuda:0', grad_fn=<SliceBackward0>)
Sample labels: tensor([0, 0, 1, 1, 0])
Epoch: 3, Average Loss: 0.08565492381049848


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

def extended_evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for text, attention_mask, label in dataloader:
            text, attention_mask, label = text.to(device), attention_mask.to(device), label.to(device)
            outputs = model(text, attention_mask=attention_mask)

            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = label.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1))
            true_labels.extend(label_ids)

    print(classification_report(true_labels, predictions))

# Call the extended evaluate function
extended_evaluate_model(model, valid_loader)

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1246
           1       0.94      0.95      0.95      1254

    accuracy                           0.95      2500
   macro avg       0.95      0.95      0.95      2500
weighted avg       0.95      0.95      0.95      2500



In [ ]:
def predict_sentiment(text, model, tokenizer):
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).cpu().numpy()[0]

    return 'Positive' if predicted_class == 1 else 'Negative'

# Example usage
sample_texts = [
    "I loved the movie! The animations and the storyline were fantastic!",
    "The product didn't meet my expectations. Quite disappointed.",
    "What a great experience, highly recommend it!",
    "Not a fan of the new update, it's quite buggy and unresponsive.",
    "This book is a masterpiece, a truly engaging story!"
]

for text in sample_texts:
    sentiment = predict_sentiment(text, model, tokenizer)
    print(f"'{text}' ... predicted sentiment: {sentiment}")

'I loved the movie! The animations and the storyline were fantastic!' ... predicted sentiment: Positive
'The product didn't meet my expectations. Quite disappointed.' ... predicted sentiment: Negative
'What a great experience, highly recommend it!' ... predicted sentiment: Positive
'Not a fan of the new update, it's quite buggy and unresponsive.' ... predicted sentiment: Negative
'This book is a masterpiece, a truly engaging story!' ... predicted sentiment: Positive


## Section 2: Summary Sentiment Analysis Model for Movie Catalogs

### Data and Training
For the sentiment analysis component, the IMDB dataset, rich in movie reviews, is ideal for understanding audience sentiments towards films. The preprocessing involved tokenization using the RoBERTa tokenizer (same approach with the earlier topic classification model) adapted for binary classification. The training process was successful, with the final average loss being reduced to approximately 0.0856.

### Results
The sentiment analysis model achieved impressive results, with precision, recall, and f1-score all around 95% for both positive and negative sentiment categories suggesting the model's effectiveness in correctly interpreting the sentiments expressed in movie reviews.

### Application for Movie Catalogs
`Sentiment Insights`: This model can analyze customer reviews and provide insights into the overall sentiment towards movies and shows. This is invaluable for streaming platforms like Netflix or community-driven platforms like YouTube to understand viewer reception.

If they were to couple a users frequently watched categories and then extrapolate positively reviewed content within that category these companies could create better reccomendations to its usersto increase their total watch time.

### Content Strategy and Recommendations:
Positive and negative sentiment analysis can influence content recommendations and acquisition strategies, highlighting titles that are well-received by audiences. It could also create an immediate feedback loop for creators on platforms hosting user-generated content.

### Example Demonstrations
The model was tested with various sample texts, ranging from positive reviews like "I loved the movie! The animations and the storyline were fantastic!" to negative ones such as "The product didn't meet my expectations. Quite disappointed." The model accurately predicted the sentiment for these samples, showcasing its practical application in real-world scenarios.

### Conclusion
The integration of both the topic classification and sentiment analysis models offers a wide ranged toolset that can be leveraged by both streaming and content platforms. While the topic classification model helps in organizing and categorizing content, the sentiment analysis model provides deeper insights into audience preferences and perceptions. This dual approach can significantly enhance content discovery, recommendation algorithms, and audience engagement strategies for movie catalog companies.

## Possible future implementations:
 A third model using the outputs of the first two to create a MVP for a "recommendation system" that could reccomend like shows or something based on a users frequently watched topics and good reviewed content within that topic

# Section 3: Model Comparison & Benchmarks

## To compare the efficacy of some different models to complete this NLP task I am going to implement the Naive Bayes Classifier (as a Baseline), RoBERTa Model, and an LSTM Classifier to complete the same task and benchmark its scores.

In [ ]:
#Step 1: Import Necessary Libraries
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

#Step 2: Load the 20 Newsgroups Dataset

from sklearn.datasets import fetch_20newsgroups

# For simplicity, let's use only a few categories
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

#Preprocess the data
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test = vectorizer.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test = newsgroups_test.target

### Naive Bayes Classifier (as a Baseline):

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       0.97      0.60      0.74       319
           1       0.96      0.89      0.92       389
           2       0.97      0.81      0.88       396
           3       0.65      0.99      0.78       398

    accuracy                           0.83      1502
   macro avg       0.89      0.82      0.83      1502
weighted avg       0.88      0.83      0.84      1502



### Implementing RoBERTa Model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW

class NewsgroupsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataset = NewsgroupsDataset(newsgroups_train.data, newsgroups_train.target, tokenizer)
test_dataset = NewsgroupsDataset(newsgroups_test.data, newsgroups_test.target, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(newsgroups_train.target_names))

# Training setup goes here
# Note: Training a transformer model on a CPU can be very slow.

# Set up GPU/CPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hyperparameters
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * epochs

# Training loop
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    # Calculate average loss over the training data
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1} / {epochs}, Average Training Loss: {avg_train_loss}")

# Save the model
model.save_pretrained('./roberta_newsgroups_model')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 / 3, Average Training Loss: 0.4850510640568297
Epoch 2 / 3, Average Training Loss: 0.1289079513386036
Epoch 3 / 3, Average Training Loss: 0.06778001671158512


### Implementing the LSTM Classifier

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from collections import Counter

class NewsgroupsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.vocab = vocab
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Tokenize and encode the text
        encoding = self.tokenizer(text)  # No need to pass vocab here, as it's included in the lambda
        encoding = encoding[:self.max_len]  # Truncate to max_length
        padding_length = self.max_len - len(encoding)
        encoding += [self.vocab.get("<PAD>")] * padding_length  # Pad with <PAD> token ID

        return {
            'input_ids': torch.tensor(encoding, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }


class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        # Take the output of the last time step
        last_time_step_out = lstm_out[:, -1, :]
        out = self.fc(last_time_step_out)
        return out

def build_vocab(texts, min_freq=1):
    # Tokenize the texts and count word frequencies
    tokenized_texts = [text.split() for text in texts]
    word_freq = Counter(word for tokens in tokenized_texts for word in tokens)

    # Build the vocabulary
    vocab = {"<PAD>": 0, "<UNK>": 1}
    idx = 2
    for word, freq in word_freq.items():
        if freq >= min_freq:
            vocab[word] = idx
            idx += 1
    return vocab

# Build the vocabulary from your training texts
vocab = build_vocab([text for text in newsgroups_train.data])


def basic_tokenizer(text, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in text.split()]


max_length = 128  # Define a suitable maximum sequence length
train_dataset = NewsgroupsDataset(newsgroups_train.data, newsgroups_train.target, lambda text: basic_tokenizer(text, vocab), max_length)
test_dataset = NewsgroupsDataset(newsgroups_test.data, newsgroups_test.target, lambda text: basic_tokenizer(text, vocab), max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Hyperparameters
embedding_dim = 128
hidden_dim = 256
vocab_size = len(vocab)
num_labels = len(newsgroups_train.target_names)

model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size, num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam as the common choice)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Number of training epochs
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader:
        # Move batch data to the device
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")


# Save the model
torch.save(model.state_dict(), 'lstm_model.pth')

Epoch 1/5, Loss: 1.3539926569226761
Epoch 2/5, Loss: 1.2850570040689389
Epoch 3/5, Loss: 1.0980495344585097
Epoch 4/5, Loss: 0.8211538023512128
Epoch 5/5, Loss: 0.5116323891135168


### Implementing the BERT model

In [ ]:

# Code for BERT model
# This would include importing the transformer library, loading a pre-trained BERT model,
# preparing the dataset for BERT, training, and evaluating the model.

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

class NewsgroupsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Use BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = NewsgroupsDataset(newsgroups_train.data, newsgroups_train.target, tokenizer)
test_dataset = NewsgroupsDataset(newsgroups_test.data, newsgroups_test.target, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Use BERT for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(newsgroups_train.target_names))

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Hyperparameters
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * epochs

# Training loop
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Backward pass
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    # Calculate average loss over the training data
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1} / {epochs}, Average Training Loss: {avg_train_loss}")

# Save the BERT model
model.save_pretrained('./bert_newsgroups_model')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 / 3, Average Training Loss: 0.3460892133322925
Epoch 2 / 3, Average Training Loss: 0.08917903684286244
Epoch 3 / 3, Average Training Loss: 0.0313308302814823


### Benchmarking and Comparing models

In [ ]:
# Code for benchmarking and comparing the three models (Naive Bayes, LSTM, BERT)
# This would include evaluating the models on the test sets, comparing their performance metrics like accuracy, F1-score, etc.
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Benchmarking and Comparing Models: Naive Bayes, LSTM, BERT
def evaluate_model(model, dataloader, device):
    model.eval()
    true_labels, predictions = [], []
    total_processed = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)

            # Assuming the model outputs logits directly
            logits = outputs
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            # Ensure logits is two-dimensional
            if logits.ndim == 1:
                logits = np.expand_dims(logits, 0)

            # Predictions
            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions.tolist())
            true_labels.extend(label_ids.tolist())

            # Diagnostic print statement
            #print(f"Processed batch with {len(label_ids)} samples, Total predictions: {len(predictions)}")

    # Diagnostic prints
    print(f"Length of true_labels: {len(true_labels)}")
    print(f"Length of predictions: {len(predictions)}")
    print(f"Sample true labels: {true_labels[:10]}")
    print(f"Sample predictions: {predictions[:10]}")

    # Simplified evaluation
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy}")

    # Attempt precision, recall, f1 calculation
    try:
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
        return accuracy, precision, recall, f1
    except ValueError as e:
        print(f"Error calculating precision, recall, f1: {e}")
        return accuracy, None, None, None


# Example: evaluate_model(lstm_model, lstm_dataloader, device)
# Load the LSTM model
lstm_model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size, num_labels)
lstm_model.load_state_dict(torch.load('lstm_model.pth'))
lstm_model.to(device)
# Evaluate LSTM Model
print("Evaluating LSTM Model...")
lstm_accuracy, lstm_precision, lstm_recall, lstm_f1 = evaluate_model(lstm_model, test_loader, device)
print(f"LSTM Model Performance:\nAccuracy: {lstm_accuracy}\nPrecision: {lstm_precision}\nRecall: {lstm_recall}\nF1 Score: {lstm_f1}")


Evaluating LSTM Model...
Length of true_labels: 1502
Length of predictions: 1502
Sample true labels: [2, 2, 2, 0, 3, 0, 1, 3, 2, 2]
Sample predictions: [1, 3, 2, 1, 1, 1, 1, 1, 3, 1]
Accuracy: 0.2496671105193076
LSTM Model Performance:
Accuracy: 0.2496671105193076
Precision: 0.25529696802807406
Recall: 0.2496671105193076
F1 Score: 0.17410529484449333


In [ ]:
# Example: evaluate_model(naive_bayes_model, naive_bayes_dataloader, device)
# Assuming y_pred_nb and y_test are available from your previous Naive Bayes model code
print("Naive Bayes Model Performance:")
print(classification_report(y_test, y_pred_nb))

# Example: evaluate_model(bert_model, bert_dataloader, device)
def evaluate_model(model, dataloader, device):
    model.eval()
    true_labels, predictions = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)

            # Extract logits from the model's output
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            # Ensure logits is two-dimensional
            if logits.ndim == 1:
                logits = np.expand_dims(logits, 0)

            # Predictions
            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions.tolist())
            true_labels.extend(label_ids.tolist())

            # Diagnostic print statement
            #print(f"Processed batch with {len(label_ids)} samples, Total predictions: {len(predictions)}")

    # Continue with evaluation
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    return accuracy, precision, recall, f1


# Load the BERT model
# bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(newsgroups_train.target_names))
# bert_model.load_state_dict(torch.load('bert_newsgroups_model'))
bert_model = BertForSequenceClassification.from_pretrained('./bert_newsgroups_model')
bert_model.to(device)

# Evaluate BERT Model
print("Evaluating BERT Model...")
bert_accuracy, bert_precision, bert_recall, bert_f1 = evaluate_model(bert_model, test_loader, device)
print(f"BERT Model Performance:\nAccuracy: {bert_accuracy}\nPrecision: {bert_precision}\nRecall: {bert_recall}\nF1 Score: {bert_f1}")

Naive Bayes Model Performance:
              precision    recall  f1-score   support

           0       0.97      0.60      0.74       319
           1       0.96      0.89      0.92       389
           2       0.97      0.81      0.88       396
           3       0.65      0.99      0.78       398

    accuracy                           0.83      1502
   macro avg       0.89      0.82      0.83      1502
weighted avg       0.88      0.83      0.84      1502



We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Evaluating BERT Model...
BERT Model Performance:
Accuracy: 0.8708388814913449
Precision: 0.8945460826111888
Recall: 0.8708388814913449
F1 Score: 0.871146121509547


In [ ]:
# Print and compare the performance metrics
# Print and compare the performance metrics of all models
# Assuming y_test and y_pred_nb are available from your Naive Bayes model evaluation
nb_accuracy = accuracy_score(y_test, y_pred_nb)
nb_precision, nb_recall, nb_f1, _ = precision_recall_fscore_support(y_test, y_pred_nb, average='weighted')

# Now print the results
print("\nModel Comparison:")
print(f"Naive Bayes - Accuracy: {nb_accuracy}, Precision: {nb_precision}, Recall: {nb_recall}, F1 Score: {nb_f1}")
print(f"LSTM - Accuracy: {lstm_accuracy}, Precision: {lstm_precision}, Recall: {lstm_recall}, F1 Score: {lstm_f1}")
print(f"BERT - Accuracy: {bert_accuracy}, Precision: {bert_precision}, Recall: {bert_recall}, F1 Score: {bert_f1}")


Model Comparison:
Naive Bayes - Accuracy: 0.8348868175765646, Precision: 0.880140416874508, Recall: 0.8348868175765646, F1 Score: 0.8368123321590524
LSTM - Accuracy: 0.2496671105193076, Precision: 0.25529696802807406, Recall: 0.2496671105193076, F1 Score: 0.17410529484449333
BERT - Accuracy: 0.8708388814913449, Precision: 0.8945460826111888, Recall: 0.8708388814913449, F1 Score: 0.871146121509547


In [ ]:
def predict_with_naive_bayes(text, model, vectorizer):
    X = vectorizer.transform([text])
    predicted = model.predict(X)
    return newsgroups_train.target_names[predicted[0]]

# Example usage for Naive Bayes
sample_text = "Global warming and environmental policy."
predicted_category = predict_with_naive_bayes(sample_text, clf_nb, vectorizer)
print(f"{sample_text} ... predicted category by Naive Bayes: {predicted_category}")
sample_text = "Medical breakthroughs in treating heart disease."
predicted_category = predict_with_naive_bayes(sample_text, clf_nb, vectorizer)
print(f"{sample_text} ... predicted category by Naive Bayes: {predicted_category}")
sample_text = "Discussing computer graphics and virtual reality systems."
predicted_category = predict_with_naive_bayes(sample_text, clf_nb, vectorizer)
print(f"{sample_text} ... predicted category by Naive Bayes: {predicted_category}")

def predict_with_lstm(text, model, tokenizer, vocab, device, max_length=128):
    # Tokenize and encode the text
    encoding = tokenizer(text, vocab)
    encoding = encoding[:max_length]
    padding_length = max_length - len(encoding)
    encoding += [vocab.get("<PAD>")] * padding_length

    # Convert to PyTorch tensor and add a batch dimension
    input_ids = torch.tensor([encoding], dtype=torch.long).to(device)

    # Prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids)
        prediction = torch.argmax(outputs, dim=1).cpu().numpy()[0]

    return newsgroups_train.target_names[prediction]

# Example usage for LSTM
sample_text = "Global warming and environmental policy."
predicted_category = predict_with_lstm(sample_text, lstm_model, basic_tokenizer, vocab, device)
print(f"{sample_text} ... predicted category by LSTM: {predicted_category}")

sample_text = "Medical breakthroughs in treating heart disease."
predicted_category = predict_with_lstm(sample_text, lstm_model, basic_tokenizer, vocab, device)
print(f"{sample_text} ... predicted category by LSTM: {predicted_category}")

sample_text = "Discussing computer graphics and virtual reality systems."
predicted_category = predict_with_lstm(sample_text, lstm_model, basic_tokenizer, vocab, device)
print(f"{sample_text} ... predicted category by LSTM: {predicted_category}")

def predict_with_bert(text, model, tokenizer, device):
    # Encode the text using the same tokenizer used during training
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',  # Ensuring consistent length
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Extract input IDs and attention masks from the encoded text
    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    # Model in evaluation mode
    model.eval()

    # Predict without computing gradients
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Extract logits
    logits = outputs.logits

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1).cpu().numpy()[0]

    return newsgroups_train.target_names[predicted_class]

# Example usage for BERT
sample_text = "Global warming and environmental policy."
predicted_category = predict_with_bert(sample_text, bert_model, tokenizer, device)
print(f"'{sample_text}' ... predicted category by BERT: {predicted_category}")
sample_text = "Medical breakthroughs in treating heart disease."
predicted_category = predict_with_bert(sample_text, bert_model, tokenizer, device)
print(f"'{sample_text}' ... predicted category by BERT: {predicted_category}")
sample_text = "Discussing computer graphics and virtual reality systems."
predicted_category = predict_with_bert(sample_text, bert_model, tokenizer, device)
print(f"'{sample_text}' ... predicted category by BERT: {predicted_category}")

Global warming and environmental policy. ... predicted category by Naive Bayes: sci.med
Medical breakthroughs in treating heart disease. ... predicted category by Naive Bayes: sci.med
Discussing computer graphics and virtual reality systems. ... predicted category by Naive Bayes: comp.graphics
Global warming and environmental policy. ... predicted category by LSTM: comp.graphics
Medical breakthroughs in treating heart disease. ... predicted category by LSTM: comp.graphics
Discussing computer graphics and virtual reality systems. ... predicted category by LSTM: comp.graphics
'Global warming and environmental policy.' ... predicted category by BERT: alt.atheism
'Medical breakthroughs in treating heart disease.' ... predicted category by BERT: sci.med
'Discussing computer graphics and virtual reality systems.' ... predicted category by BERT: comp.graphics


# Section 3: Model Comparison & Benchmarks
## Overview
Section 3 involved benchmarking different models for NLP tasks. Specifically, it compared the Naive Bayes Classifier (Baseline), RoBERTa Model, LSTM Classifier, and BERT Model on the 20 Newsgroups dataset. The goal was to evaluate and contrast their performance in text classification tasks.

## Implementation and Results
### Naive Bayes Classifier (Baseline):

`Approach`: Utilized a TfidfVectorizer for feature extraction and MultinomialNB for classification.

`Results`: Achieved an overall accuracy of 83.49% with high precision and recall in specific categories. However, the model showed limitations in handling some categories, particularly 'sci.med'.

### RoBERTa Model:

`Approach`: Employed a pre-trained RoBERTa model fine-tuned on the dataset.

`Results`: The RoBERTa model showed significant improvement across epochs, with the final average training loss being 0.0678. This model's performance was not explicitly mentioned, but typically, RoBERTa models perform well in text classification tasks.

### LSTM Classifier:

`Approach`: Implemented an LSTM model with custom tokenization and vocabulary building.

`Results`: The LSTM model's performance was notably lower, with an accuracy of 24.97% and lower precision, recall, and F1 scores. This suggests that the model struggled with the complexity of the dataset or needed further tuning and training.

### BERT Model:

`Approach`: Utilized BERT for sequence classification, fine-tuned on the dataset.

`Results`: BERT achieved a high accuracy of 87.08%, with strong precision, recall, and F1 scores, indicating its effectiveness in handling complex text classification tasks.

### Predictive Testing

`Naive Bayes`: Demonstrated robust performance in certain categories but lacked in others.

`RoBERTa`: Although specific performance metrics were not provided, RoBERTa generally shows strong capabilities in text classification.

`LSTM`: Underperformed, likely due to its simpler architecture and potential issues with training and data processing.

`BERT`: Excelled in classification tasks expected especially if u factor its status in NLP.

### Conclusion and Application

Naive Bayes serves as a competent baseline but may not handle complex categorizations well.
RoBERTa and BERT are more suited for intricate tasks, with BERT slightly outperforming RoBERTa in this case.
The LSTM model, while valuable in sequential data, might require more nuanced tuning and data preparation to compete with transformer-based models in text classification tasks.

Moral of the story is if I was a company like Netflix, I would use a model like Bert or Roberta for the better accuracy and user experience even though it takes longer to train compared to a more naive or simple LSTM / Bayes model.